In [1]:
import time
import numpy as np
import pandas as pd
from sklearn import svm
from sklearn.preprocessing import normalize
from sklearn.model_selection import train_test_split

In [2]:
# Thiết lập tham số
path_file = "./heart.csv"
distance_result = 0.5

In [3]:
def read_file(path):
    # read file
    #csv = np.genfromtxt(path, delimiter=",")
    #csv = csv[1:,1:]
    data = pd.read_csv(path).as_matrix()
    data[data == "b"] = 1
    data[data == "a"] = 0
    data = data[0:,1:]
    return data

In [4]:
csv = read_file(path_file)
csv

/Users/admin/.conda/envs/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py:5: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """


array([[70, 1, 4, ..., 3, 3, 1],
       [67, 0, 3, ..., 0, 7, 0],
       [57, 1, 2, ..., 0, 7, 1],
       ...,
       [56, 0, 2, ..., 0, 3, 0],
       [57, 1, 4, ..., 0, 6, 0],
       [67, 1, 4, ..., 3, 3, 1]], dtype=object)

In [5]:
# Chuẩn hóa min-max trong khoảng tùy chọn
# Chú ý chuẩn hóa theo từng cột để ứng với từng thuộc tính
def scale(X, x_min, x_max):
    nom = (X-X.min(axis=0))*(x_max-x_min)
    denom = X.max(axis=0) - X.min(axis=0)
    denom[denom==0] = 1
    return x_min + nom/denom 

In [6]:
csv = scale(csv, 0, 1)

In [7]:
# Trả về các vector thuộc tính
def matrix_atribute(csv):
    m_atribute = csv[:,:].T
    m_d = csv[:,-1]
    return m_atribute, m_d

In [8]:
# Thuc thi
m_atribute, m_d = matrix_atribute(csv)

In [9]:
m_atribute

array([[0.8541666666666666, 0.7916666666666666, 0.5833333333333334, ...,
        0.5625, 0.5833333333333334, 0.7916666666666666],
       [1.0, 0.0, 1.0, ..., 0.0, 1.0, 1.0],
       [1.0, 0.6666666666666666, 0.3333333333333333, ...,
        0.3333333333333333, 1.0, 1.0],
       ...,
       [1.0, 0.0, 0.0, ..., 0.0, 0.0, 1.0],
       [0.0, 1.0, 1.0, ..., 0.0, 0.75, 0.0],
       [1.0, 0.0, 1.0, ..., 0.0, 0.0, 1.0]], dtype=object)

In [10]:
m_atribute.shape

(14, 270)

In [11]:
def member(x,y):
    return round(1.0-abs(x - y),2)

In [12]:
def f_relation_matrix(arr_c):
    matrix_0 = np.zeros((len(arr_c), len(arr_c)), dtype=np.float32)
    for i in range(len(arr_c)):
        for j in range(len(arr_c)):
            matrix_0[i][j] = member(arr_c[i], arr_c[j])
    return matrix_0

In [13]:
def f_matrix_relation_atribute(f_m_atribute):
    f_matrix_relation_atribute = []
    for i in range(f_m_atribute.shape[0]):
        f_mra = f_relation_matrix(m_atribute[i])
        f_matrix_relation_atribute.append(f_mra)
    f_matrix_relation_atribute = np.array(f_matrix_relation_atribute)
    return f_matrix_relation_atribute

In [14]:
f_matrix_relation_atribute = f_matrix_relation_atribute(m_atribute)
f_matrix_relation_atribute.shape

(14, 270, 270)

In [15]:
def f_matrix_intersect_relation(m1, m2):
    f_matrix_intersect_relation = np.zeros_like(m1)
    for i in range(len(f_matrix_intersect_relation[0])):
        for j in range(len(f_matrix_intersect_relation[0])):
            if m1[i][j] < m2[i][j]:
                f_matrix_intersect_relation[i][j] = m1[i][j]
            else:
                f_matrix_intersect_relation[i][j] = m2[i][j]
    return f_matrix_intersect_relation

In [16]:
# cal c1 in d, c2 in d, cn in d of fuzzy
def f_matrix_intersect_d(f_mra):
    f_matrix_intersect_d = []
    for i in range(f_mra.shape[0]):
        mtg = f_matrix_intersect_relation(f_mra[i],f_mra[-1])
        f_matrix_intersect_d.append(mtg)
    f_matrix_intersect_d = np.array(f_matrix_intersect_d)
    return f_matrix_intersect_d

In [17]:
f_matrix_intersect_d = f_matrix_intersect_d(f_matrix_relation_atribute)
f_matrix_intersect_d.shape

(14, 270, 270)

In [18]:
def cal_lamda(m_atribute):
    print(m_atribute.shape)
    ts = []
    #lamda = []
    #mab = m_atribute[:-1,:]
    #print(mab.shape)
    # ddof for N-1
    xm = np.std(m_atribute, axis=1, ddof=1, dtype=np.float32)
    
    cl = 0
    for a in f_matrix_intersect_d:
        for b in a:
            cl += np.sum(b)
        ts.append(cl)
    ms = 0
    for v in f_matrix_relation_atribute[-1]:
        ms += np.sum(v)
    ts = np.array(ts)
    vector_beta = ts / ms
    print(xm.shape)
    print(vector_beta.shape)
    lamda = [1 if xm[i] == 0 else vector_beta[i]/xm[i] for i in range(len(xm))]
    lamda = np.array(lamda)
    return lamda

In [19]:
lamda = cal_lamda(m_atribute)
lamda

(14, 270)
(14,)
(14,)


array([ 4.16396503,  2.94832984,  6.61763232, 17.29197626, 32.10971311,
       12.74459805, 10.12488322, 33.2205901 , 13.85101438, 39.81825244,
       26.30208264, 28.08050912, 19.54414647, 21.05578232])

In [20]:
def non_member(m, el_lam):
    lamda = 1.0
    return round((1.0 - m)/(1.0 + el_lam * m),2)

In [21]:
def if_relation_matrix(arr_c, el_lam):
    if_relation_matrix = np.zeros((len(arr_c), len(arr_c)), dtype="f,f")
    for i in range(len(arr_c)):
        for j in range(len(arr_c)):
            if_relation_matrix[i][j][0] = member(arr_c[i], arr_c[j])
            if_relation_matrix[i][j][1] = non_member(if_relation_matrix[i][j][0], el_lam)
    return if_relation_matrix

In [22]:
# Tinh toan ma tran quan he theo tung atribute cua IF
def if_matrix_relation_atribute(m_atribute, lamda):
    if_matrix_relation_atribute = []
    for i in range(m_atribute.shape[0]):
        mra = if_relation_matrix(m_atribute[i], lamda[i])
        if_matrix_relation_atribute.append(mra)
    if_matrix_relation_atribute = np.array(if_matrix_relation_atribute)
    return if_matrix_relation_atribute

In [23]:
if_matrix_relation_atribute = if_matrix_relation_atribute(m_atribute, lamda)
if_matrix_relation_atribute.shape

(14, 270, 270)

In [24]:
def if_matrix_intersect_relation(m1, m2):
    matrix_init = np.zeros_like(m1)
    for i in range(len(m1[0])):
        for j in range(len(m1[0])):
            if m1[i][j][0] < m2[i][j][0]:
                matrix_init[i][j] = m1[i][j]
            else:
                matrix_init[i][j] = m2[i][j]
    return matrix_init

In [25]:
# cal c1 in d, c2 in d, cn in d
def if_matrix_intersect_d(mra):
    if_matrix_intersect_d = []
    for i in range(mra.shape[0]):
        mtg = if_matrix_intersect_relation(mra[i],mra[-1])
        if_matrix_intersect_d.append(mtg)
    return if_matrix_intersect_d[:-1]

In [26]:
if_matrix_intersect_d = if_matrix_intersect_d(if_matrix_relation_atribute)
if_matrix_intersect_d

[array([[(1.  , 0.  ), (0.  , 1.  ), (0.73, 0.07), ..., (0.  , 1.  ),
         (0.  , 1.  ), (0.94, 0.01)],
        [(0.  , 1.  ), (1.  , 0.  ), (0.  , 1.  ), ..., (0.77, 0.05),
         (0.79, 0.05), (0.  , 1.  )],
        [(0.73, 0.07), (0.  , 1.  ), (1.  , 0.  ), ..., (0.  , 1.  ),
         (0.  , 1.  ), (0.79, 0.05)],
        ...,
        [(0.  , 1.  ), (0.77, 0.05), (0.  , 1.  ), ..., (1.  , 0.  ),
         (0.98, 0.  ), (0.  , 1.  )],
        [(0.  , 1.  ), (0.79, 0.05), (0.  , 1.  ), ..., (0.98, 0.  ),
         (1.  , 0.  ), (0.  , 1.  )],
        [(0.94, 0.01), (0.  , 1.  ), (0.79, 0.05), ..., (0.  , 1.  ),
         (0.  , 1.  ), (1.  , 0.  )]], dtype=[('f0', '<f4'), ('f1', '<f4')]),
 array([[(1., 0.), (0., 1.), (1., 0.), ..., (0., 1.), (0., 1.), (1., 0.)],
        [(0., 1.), (1., 0.), (0., 1.), ..., (1., 0.), (0., 1.), (0., 1.)],
        [(1., 0.), (0., 1.), (1., 0.), ..., (0., 1.), (0., 1.), (1., 0.)],
        ...,
        [(0., 1.), (1., 0.), (0., 1.), ..., (1., 0.), (0., 1.

In [27]:
# cal C 
def if_matrix_atribute_C(mra):
    if_matrix_multi_atribute_intersect = []
    for i in range(mra.shape[0] - 1):
        matrix_2 = if_matrix_intersect_relation(mra[i], mra[i+1])
        if_matrix_multi_atribute_intersect.append(matrix_2)
    if_matrix_atribute_C = np.array(if_matrix_multi_atribute_intersect)[-2]
    return if_matrix_atribute_C

In [28]:
if_matrix_atribute_C = if_matrix_atribute_C(if_matrix_relation_atribute)
if_matrix_atribute_C.shape

(270, 270)

In [29]:
def cal_len(arr):
    return np.round(float(np.sum([(1+e[0]-e[1])/2 for e in arr])),2)

In [30]:
def distance_measure(A,d):
    tg = if_matrix_intersect_relation(A,d)
    g = [round(cal_len(A[i])-cal_len(tg[i]),3) for i in range(len(A[0]))]
    return round(1/(len(A[0]) * len(A[0])) * np.sum(g),2)

In [31]:
def matrix_distance_measure(mra):
    vector_distance_measure = []
    for i in range(mra.shape[0]):
        dis = distance_measure(mra[i], mra[-1])
        vector_distance_measure.append(dis)
    vector_distance_measure = np.array(vector_distance_measure)
    return vector_distance_measure

In [32]:
matrix_distance_measure = matrix_distance_measure(if_matrix_relation_atribute)
matrix_distance_measure

array([0.42, 0.26, 0.36, 0.44, 0.46, 0.37, 0.24, 0.44, 0.23, 0.44, 0.38,
       0.37, 0.2 , 0.  ])

In [33]:
distance_measure_C = distance_measure(if_matrix_atribute_C, if_matrix_relation_atribute[-1])
distance_measure_C

0.15

In [34]:
index = np.where(matrix_distance_measure[:-1] == min(matrix_distance_measure[:-1]))[0][0]
index

12

In [35]:
def filter_step(distance_measure_C,matrix_distance_measure):
    C = [i for i in range(0,if_matrix_relation_atribute.shape[0] - 1)]
    d = 0.5
    step = 1
    W = []
    B = []
    ind = np.where(matrix_distance_measure[:-1] == min(matrix_distance_measure[:-1]))[0][0]
    B.append(ind)
    W.append(B)
    C.remove(ind)
    print(f"Step {step} W = ", W)
    print(f"d = ", d)
    mtg = if_matrix_relation_atribute[index]
    while d > distance_measure_C:
        step +=1
        tg = dict()
        for i in C:
            ints = if_matrix_intersect_relation(mtg, if_matrix_relation_atribute[i])     
            tg[i] = ints
        distance = [distance_measure(tg[x], if_matrix_relation_atribute[-1]) for x in tg.keys()]

        d = np.min(distance)
        ind = C[np.argmin(distance)]
        mtg = tg[ind]
        B = B[:] + [ind]
        W.append(B)
        print(f"Step {step} W = ", W)
        print(f"d = ", d)
    return W

In [ ]:
h = filter_step(distance_measure_C,matrix_distance_measure)

Step 1 W =  [[12]]
d =  0.5
